In [1]:
# boiler plate to allow running in colab, can ignore if running locally
import subprocess
try:
    import tailnflows
except ModuleNotFoundError:
    # need to build environment
    print('installing tailnflows environment...')
    subprocess.run(['pip', 'install', 'git+https://github.com/Tennessee-Wallaceh/tailnflows'])
    import tailnflows
    from tailnflows.utils import configure_colab_env
    configure_colab_env() # can take a while
    import torch
    torch.set_default_device('cuda')

In [3]:
import torch
from tailnflows.models.flows import (
    ModelUse, TTF_m, gTAF, mTAF, _df_to_tailp
)

models = {
    'TTF_m_fixed': lambda dim, dfs: TTF_m(
        dim,
        ModelUse.variational_inference,
        model_kwargs=dict(
            tail_bound=5.,
            rotation=False,
            num_bins=8,
            pos_tail_init=_df_to_tailp(dfs),
            neg_tail_init=_df_to_tailp(dfs),
            fix_tails=True,
        )
    ),
    'TTF_m': lambda dim, _: TTF_m(
        dim,
        ModelUse.variational_inference,
        model_kwargs=dict(
            tail_bound=5.,
            rotation=False,
            num_bins=8,
        )
    ),
    'mTAF': lambda dim, dfs: mTAF(
        dim,
        ModelUse.variational_inference,
        model_kwargs=dict(
            tail_bound=5.,
            rotation=False,
            tail_init=torch.tensor(dfs),
            num_bins=8
        )
    ),
    'gTAF': lambda dim, _: gTAF(
        dim, 
        ModelUse.variational_inference, 
        model_kwargs=dict(tail_bound=5., rotation=False, num_bins=8)
    ),
}

In [4]:
from tailnflows.train.variational_fit import train
from tailnflows.targets.heavy_tailed_nuisance import log_density
from tailnflows.experiments.utils import add_raw_data
import itertools

# experiment parameters
target_dims = [5]
nuisance_dfs = [1., 2., 30.]
repeats = 5
experiment_params = itertools.product(range(repeats), target_dims, nuisance_dfs)

for repeat, dim, nuisance_df in experiment_params:
    print('=' * 20, f'v={nuisance_df}, dim={dim}', '=' * 20)
    dfs = [nuisance_df] * dim # for shift data set all dfs are the same
    for label, model_fcn in models.items():
          torch.manual_seed(repeat)

          model = model_fcn(dim, dfs).to(torch.float32)

          losses, metrics = train(
            model,
            lambda x: log_density(x, heavy_df=nuisance_df),
            lr=1e-3,
            num_epochs=300,
            batch_size=100,
            label=label,
            seed=repeat,
          )

          add_raw_data('vi_newrun', label, (metrics.ess, metrics.psis_k, dim, nuisance_df))

/home/th17628/miniconda3/envs/tailnflows/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


==================== v=1.0, dim=5 ====================


100%|██████████| 300/300 [00:05<00:00, 56.21it/s, elbo=-0.788, tst_ess=0.001, tst_psis=0.821, model=gTAF]


==================== v=2.0, dim=5 ====================


100%|██████████| 300/300 [00:05<00:00, 54.05it/s, elbo=-0.276, tst_ess=0.213, tst_psis=0.837, model=gTAF]


...

==================== v=1.0, dim=5 ====================


100%|██████████| 300/300 [00:06<00:00, 48.56it/s, elbo=-1.721, tst_ess=0.002, tst_psis=0.839, model=gTAF]


==================== v=2.0, dim=5 ====================


 36%|███▌      | 108/300 [00:02<00:03, 52.99it/s, neg_elbo=0.94, model=TTF_m]


KeyboardInterrupt: 